# Mphasis HyperGraf Hotel Reservation Cancellation Predictor

Mphasis HyperGraf Hotel Reservation Cancellation Predictor is an Ensemble Machine Learning algorithm-based solution which predicts the probability of guests cancelling their hotel reservations. Cancellation of scheduled hotel stays is a significant challenge for hospitality companies.Cancellations lead to erroneous demand estimation, room pricing and revenue management. This solution predicts the likelihood of guests cancelling their hotel reservations based on guests' booking information. The Solution assists hospitality companies to maximize occupancy and revenue per available room.

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

This sample notebook shows you how to deploy Mphasis HyperGraf Hotel Reservation Cancellation Predictor using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Mphasis HyperGraf Hotel Reservation Cancellation Predictor. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Mphasis HyperGraf Hotel Reservation Cancellation Predictor
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/hotel-cancel-v1'

In [2]:
import pandas as pd
import numpy as np
import json
import os
import boto3
from zipfile import ZipFile
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from IPython.display import Image, display

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='hotel-cancel'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.

model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [ ]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

#### Instructions

1) Supported content types: 'csv' file only

2) Mandatory fields: Guest_Id, hotel_location_type,lead_time,  expected_arrival_month, expected_weekend_stays, expected_weekday_stays, adults, children, babies,meal_plan , guest_market_segment, repeat_guest, previous_cancellations, previous_bookings_not_canceled, booking_modifications,reservation_queue,average_daily_rate, required_car_parking_spaces, special_requests.

3) Input field descriptions:

* Guest_Id: Numeric or alpha-numeric value to uniquely identify a guest. For example, ‘G01’. (Categorical)
 
* hotel_location_type:  If the location of the hotel is in city then value is ‘City Destination’, Else. if the location is a holiday spot then value is ‘Holiday Destination’. (Categorical)
 
* lead_time: Number of days between the date of reservation and the expected date of arrival. For example, If confirmation date is July 1 and appointment date is July 30, then Lead_Time value is ‘29’. (Numeric)
 
* expected_arrival_month:  Month of expected arrival date. The values can be  ‘January’ , February‘, ‘March’,’ April’, ‘May’, ’June’,  ’July’, ‘August’, ’September’, ’October’, ’November’, ’December’. (Categorical)
 
* expected_weekend_stays: Number of weekend nights (Saturday or Sunday) the guest booked to stay at the hotel. (Numeric)
 
* expected_weekday_stays: Number of week day nights (Monday to Friday) the guest booked to stay at the hotel. (Numeric)
 
* adults : Number of adults. (Numeric)
 
* children :  Number of children. (Numeric)

* babies : Number of babies. (Numeric)
 
* meal_plan:  Type of meal booked. 

    If the meal plan is not confirm, the value is ‘Undefined’, Else.

    If the meal plan is Self-Catering, the value is ‘SC’, Else.

    If the meal plan is of type Bed and Breakfast, the value is ‘BB’, Else.

    If the meal plan is of type Half Board (breakfast + 1 other meal), the value is ‘HB’, Else.
    
    If the meal plan is of type Full Board (breakfast + lunch+ dinner) , the value is ‘FB’. (Categorical)
 
* guest_market_segment : segment to which the guest belong to.

    If the market-segment is Online Travel Agent/ Travel Operator, then the value is ‘Online TA’, Else.
    
    If the market-segment is Offline Travel Agent/ Travel Operator, then the value is ‘Offline TA’, Else.

    If the guest market-segment is Groups type, then the value is ‘Groups’, Else.

    If the guest market-segment is Direct type, the value is ‘Direct’, Else.

    If the guest market-segment is Corporate type, the value is ‘Corporate’, Else.

    If the guest market-segment is Aviation type, the value is ‘Aviation’, Else.

    If the guest market-segment is Complimentary type, the value is ‘Complimentary’, Else.

    If the guest market-segment is Undefined, the value is ‘Undefined’. (Categorical)
 
* repeat_guest - If a guest has previous reservations in the hotel or stayed at the hotel in past, the value is ‘Y’ Else. the value is ‘N’. (Categorical)
 
* previous_cancellations - Number of previous reservations that were cancelled by the guest prior to the current reservation. (Numerical)
 
* previous_bookings_not_canceled -Number of previous reservations not cancelled by the guest prior to the current reservation. (Numerical)

* booking_modifications – Number of amendments made to the reservation. (Numerical)

* reservation_queue– Number of days the booking was in the waiting list before the reservation was confirmed. (Numerical)

* average_daily_rate – total booking amount divided by the number of days of booking, in € (Euro).  (Numerical)

* required_car_parking_spaces – Number of car parking spaces required by the guest. (Numerical)

* special_requests – Number of special requests made by the guest.  (Numerical)


In [7]:
file_name = 'sample_input.csv'

#### C. Perform real-time inference

In [8]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/csv' --region us-east-2 output.csv

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

* Supported content types: 'csv' files only. 

* Output file columns: Guest_Id & Cancellation_Probability. 

* The Cancellation_Probability corresponding to a Guest_Id is the probability of guests cancelling their reservations. 


In [9]:
file_path = os.getcwd()
output_df_path = os.path.join(file_path, 'output.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)
output_df.head()

Sample Output:


,Guest_Id,Cancellation_Probability
0,1001,0.267
1,1002,0.879
2,1003,0.354
3,1004,0.583
4,1005,0.315


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [10]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

In [ ]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
#output is available on following path
transformer.output_path

In [13]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('output2.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+'/sample_input.csv.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [14]:
file_path = os.getcwd()
output_df_path = os.path.join(file_path, 'output2.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)
output_df.head()

Sample Output:


,Guest_Id,Cancellation_Probability
0,1001,0.267
1,1002,0.879
2,1003,0.354
3,1004,0.583
4,1005,0.315


### 4. Clean-up

#### A. Delete the model

In [15]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.